In [1]:
import torch 
import torchvision
import torch.nn as nn
from torch.autograd import Variable 

In [2]:
#import torchvision.datasets as datasets
import numpy as np

In [3]:
from torch.utils.data import DataLoader,Dataset

In [4]:
import torchvision.transforms as transforms

In [5]:
import torch.optim as optim

In [6]:
# import torchvision
# import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
# from torch.utils.data import Dataset, DataLoader
# import numpy as np

# Transformations
RC   = transforms.RandomCrop(32, padding=4)
RHF  = transforms.RandomHorizontalFlip()
RVF  = transforms.RandomVerticalFlip()
NRM  = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
TT   = transforms.ToTensor()
TPIL = transforms.ToPILImage()

# Transforms object for trainset with augmentation
transform_with_aug = transforms.Compose([TPIL, RC, RHF, TT, NRM])
# Transforms object for testset with NO augmentation
transform_no_aug   = transforms.Compose([TT, NRM])

# Downloading/Louding CIFAR10 data
trainset  = CIFAR10(root='/DATA/Ravi10/Data/CIFAR_10/', train=True , download=True)#, transform = transform_with_aug)
testset   = CIFAR10(root='/DATA/Ravi10/Data/CIFAR_10/', train=False, download=True)#, transform = transform_no_aug)
classDict = {'plane':0, 'car':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}

# Separating trainset/testset data/label
x_train  = trainset.data
x_test   = testset.data
y_train  = trainset.targets
y_test   = testset.targets

# Define a function to separate CIFAR classes by class index

def get_class_i(x, y, i):
    """
    x: trainset.train_data or testset.test_data
    y: trainset.train_labels or testset.test_labels
    i: class label, a number between 0 to 9
    return: x_i
    """
    # Convert to a numpy array
    y = np.array(y)
    # Locate position of labels that equal to i
    pos_i = np.argwhere(y == i)
    # Convert the result into a 1-D list
    pos_i = list(pos_i[:,0])
    # Collect all data that match the desired label
    x_i = [x[j] for j in pos_i]
    
    return x_i

class DatasetMaker(Dataset):
    def __init__(self, datasets, transformFunc = transform_no_aug):
        """
        datasets: a list of get_class_i outputs, i.e. a list of list of images for selected classes
        """
        self.datasets = datasets
        self.lengths  = [len(d) for d in self.datasets]
        self.transformFunc = transformFunc
    def __getitem__(self, i):
        class_label, index_wrt_class = self.index_of_which_bin(self.lengths, i)
        img = self.datasets[class_label][index_wrt_class]
        img = self.transformFunc(img)
        return img, class_label

    def __len__(self):
        return sum(self.lengths)
    
    def index_of_which_bin(self, bin_sizes, absolute_index, verbose=False):
        """
        Given the absolute index, returns which bin it falls in and which element of that bin it corresponds to.
        """
        # Which class/bin does i fall into?
        accum = np.add.accumulate(bin_sizes)
        if verbose:
            print("accum =", accum)
        bin_index  = len(np.argwhere(accum <= absolute_index))
        if verbose:
            print("class_label =", bin_index)
        # Which element of the fallent class/bin does i correspond to?
        index_wrt_class = absolute_index - np.insert(accum, 0, 0)[bin_index]
        if verbose:
            print("index_wrt_class =", index_wrt_class)

        return bin_index, index_wrt_class

# ================== Usage ================== #

# Let's choose cats (class 3 of CIFAR) and dogs (class 5 of CIFAR) as trainset/testset
my_trainset = \
    DatasetMaker(
        [get_class_i(x_train, y_train, classDict['plane']), get_class_i(x_train, y_train, classDict['car']),get_class_i(x_train, y_train, classDict['bird']),get_class_i(x_train, y_train, classDict['cat']),get_class_i(x_train, y_train, classDict['deer'])],
        transform_with_aug
    )
    
    


Files already downloaded and verified
Files already downloaded and verified


In [7]:
my_testset = DatasetMaker(
        [get_class_i(x_train, y_train, classDict['plane']), get_class_i(x_train, y_train, classDict['car']),get_class_i(x_train, y_train, classDict['bird']),get_class_i(x_train, y_train, classDict['cat']),get_class_i(x_train, y_train, classDict['deer'])],
        transform_no_aug
    )

kwargs = {'num_workers': 2, 'pin_memory': False}

# Create datasetLoaders from trainset and testset
trainsetLoader   = DataLoader(my_trainset, batch_size=64, shuffle=True , **kwargs)
testsetLoader    = DataLoader(my_testset , batch_size=64, shuffle=False, **kwargs)

In [8]:
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet,self).__init__()
        self.layer1 = nn.Conv2d(3,6,kernel_size=5,stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer2 = nn.Conv2d(6,16,kernel_size=5,stride=1)
        #self.layer3 = nn.Conv2d(16,120,kernel_size=5,stride=1)
        self.relu = nn.ReLU()
        self.smax = nn.Softmax()
        self.fc1 = nn.Linear(400,120)
        self.fc2 = nn.Linear(120,84)
        self.out = nn.Linear(84,5)
    def forward(self,x):
        a = self.relu(self.layer1(x))
        a = self.relu(self.pool1(a))
        a = self.relu(self.layer2(a))
        a = self.relu(self.pool1(a))
        #a = self.layer3(a)
        #print(a.size)
        b = a.view(a.shape[0],-1)
        b = self.relu(self.fc1(b))
        b = self.relu(self.fc2(b))
        out = self.out(b)
        return out

In [9]:
device='cuda:0'

In [10]:
model = Lenet().to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.1,momentum=0.5)

In [12]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [13]:
def train(epoch):
    model.train()
    for i,(inputs,targets) in enumerate(trainsetLoader):
        inputs = Variable(inputs)
        inputs = inputs.to(device)
        targets = Variable(targets)
        targets = targets.to(device)
        #print(targets.shape)
        preds = model(inputs)
        optimizer.zero_grad()
        loss = criterion(preds,targets)
        loss.backward()
        optimizer.step()
        if i%100 == 0:
            temp = 100.*i*len(inputs)/len(trainsetLoader.dataset)
            print('Epoch:{} [{}/{}] {:.0f}% Loss:{:.4f}'.format(epoch,i*len(inputs),len(trainsetLoader.dataset),temp,loss))

In [14]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs,targets in testsetLoader:
            inputs = Variable(inputs)
            inputs = inputs.to(device)
            targets = Variable(targets)
            targets = targets.to(device)
            preds = model(inputs)
            test_loss += criterion(preds,targets)
            label = torch.max(preds,1)[1]
            correct += label.eq(targets.view_as(label)).cpu().sum()
        test_loss /= len(testsetLoader.dataset)
        acc = 100.*correct/len(testsetLoader.dataset)
        print('\nTest Error:{:.4f} Correct:{} Accuracy:{:.4f}'.format(test_loss,correct,acc))


In [15]:
for i in range(1,40):
  #print(torch.cuda.is_available())
    train(i)
    test()
    scheduler.step()

Epoch:1 [0/25000] 0% Loss:1.6008
Epoch:1 [6400/25000] 26% Loss:1.3381
Epoch:1 [12800/25000] 51% Loss:1.0976
Epoch:1 [19200/25000] 77% Loss:1.2852

Test Error:0.0156 Correct:14816 Accuracy:59.2640
Epoch:2 [0/25000] 0% Loss:1.0151
Epoch:2 [6400/25000] 26% Loss:1.1768
Epoch:2 [12800/25000] 51% Loss:1.0334
Epoch:2 [19200/25000] 77% Loss:0.8002

Test Error:0.0154 Correct:14606 Accuracy:58.4240
Epoch:3 [0/25000] 0% Loss:1.0519
Epoch:3 [6400/25000] 26% Loss:0.9489
Epoch:3 [12800/25000] 51% Loss:1.0462
Epoch:3 [19200/25000] 77% Loss:0.9706

Test Error:0.0139 Correct:16289 Accuracy:65.1560
Epoch:4 [0/25000] 0% Loss:0.8400
Epoch:4 [6400/25000] 26% Loss:0.9293
Epoch:4 [12800/25000] 51% Loss:0.9528
Epoch:4 [19200/25000] 77% Loss:1.1105

Test Error:0.0147 Correct:15752 Accuracy:63.0080
Epoch:5 [0/25000] 0% Loss:0.9509
Epoch:5 [6400/25000] 26% Loss:0.8358
Epoch:5 [12800/25000] 51% Loss:0.8524
Epoch:5 [19200/25000] 77% Loss:0.9723

Test Error:0.0141 Correct:16142 Accuracy:64.5680
Epoch:6 [0/25000] 0%

In [70]:
def conf_creator():
    device='cuda:0'
    model.eval()
    test_loss = 0
    correct = 0
    matrix = np.zeros((5,5))
    for inputs,targets in testsetLoader:
        inputs = Variable(inputs)
        inputs = inputs.to(device)
        targets = Variable(targets)
        targets = targets.to(device)
        preds = model(inputs)
        
        label = torch.max(preds,1)[1]
        l = list(targets.shape)
        
        i = 0
        while i < l[0]:
            matrix[targets[i]][label[i]] += 1 
            #print(matrix)
            i = i+1
    print(matrix)
        

In [71]:
conf_creator()

[[4184.  272.  298.  147.   99.]
 [ 144. 4736.   34.   66.   20.]
 [ 405.   92. 3417.  599.  487.]
 [ 217.  105.  489. 3805.  384.]
 [ 206.   57.  550.  551. 3636.]]
